# 🍱 공무원 맛집 추천 시스템 머신러닝 분류 모델 개선

이 노트북은 서울시 공무원 업무추진비 데이터를 기반으로 머신러닝 분류 모델을 학습하여 맛집 카테고리를 예측하고, 해당 카테고리에 속하는 추천 음식점 이름을 출력합니다.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


## 1. 데이터 불러오기

In [5]:
# CSV 파일 경로에 맞게 수정 필요C:\null4\GongPick\data\프렌차이즈_구추출_결과.csv
df = pd.read_csv("GongPick\data\프렌차이즈_구추출_결과.csv", encoding='utf-8')
df.head()


,집행일시,시간,집행목적,사용장소,인원,금액,계절,1인당비용,중복,주소지,업종 대분류,업종 중분류,업종 소분류,업종코드,점저,구
0,2023-10-31 0:00,오전 9:28:00,국가지정문화재 보수정비 현장검토 관련 간담회,연회다원,3,16500,가을,5500.0,2,강남구 봉은사로 531,음식점,카페,NaN,CE7,점심,강남구
1,2023-10-30 0:00,오후 12:14:00,부서 전입 직원 격려 간담회,한와담 광화문점,5,75000,가을,15000.0,130,중구 세종대로21길 40,음식점,한식,"육류,고기",FD6,점심,중구
2,2023-10-27 0:00,오후 7:17:00,생생문화재 활용사업 추진 관련 간담회,무교동낙지,4,49000,가을,12250.0,653,중구 덕수궁길 7,음식점,한식,"해물,생선",FD6,저녁,중구
3,2023-10-24 0:00,오후 12:33:00,문화재 지정조사 실시 관련 간담회,남산복집,4,60000,가을,15000.0,2,중구 무교동 1번지 신라 1호,음식점,한식,"해물,생선",FD6,점심,중구
4,2023-10-23 0:00,오후 7:26:00,남산봉수의식 추진 관련 간담회,종로황소곱창,4,61000,가을,15250.0,2,종로구 우정국로 2길 31,음식점,한식,"육류,고기",FD6,저녁,종로구


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report


# 사용할 컬럼 정의
features = ['인원', '계절', '점저', '1인당비용','업종 중분류','구']
label = '사용장소'

# 결측값 제거
df_clean = df[features + [label]].dropna()

# 사용장소 기준 상위 20개만 필터링
top_places = df_clean[label].value_counts().nlargest(20).index
df_filtered = df_clean[df_clean[label].isin(top_places)]

# 특성과 타깃 분리
X = df_filtered[features]
y = df_filtered[label]

# 수치형/범주형 특성 정의
numeric_features = ['인원', '1인당비용']
categorical_features = ['계절', '점저', '업종 중분류','구']

# 전처리 파이프라인
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# 전체 파이프라인 구성
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
pipeline.fit(X_train, y_train)

# 평가
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))




               precision    recall  f1-score   support

         곰국시집       0.00      0.00      0.00       124
         구이구이       0.20      0.46      0.28       177
           대복       0.00      0.00      0.00        79
         라칸티나       1.00      1.00      1.00        77
        무교동낙지       0.00      0.00      0.00        76
        무교소호정       0.25      0.05      0.09        95
          배수사       0.42      0.29      0.34       108
         배재반점       0.56      0.76      0.65       109
          복성각       0.40      0.21      0.27        81
      브이아이피참치       0.34      0.15      0.21       159
          삼우정       0.21      0.73      0.33       232
서울시청 카페테리아 마루       1.00      1.00      1.00        84
         오리마당       0.07      0.01      0.01       177
         우도일식       0.50      0.81      0.62       201
     월매네남원추어탕       0.18      0.37      0.24        92
        장안삼계탕       0.00      0.00      0.00        84
          잼배옥       0.00      0.00      0.00        75
  주식회사  부

c:\Users\User\anaconda3\envs\gongpick-env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\anaconda3\envs\gongpick-env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\anaconda3\envs\gongpick-env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()}

In [7]:
# 예측 예시
example = pd.DataFrame([{
    '인원': 4,
    '계절': '가을',
    '점저': '점심',
    '1인당비용': 30000,
    '업종 중분류' : '일식',
    '구': '강남구'
}])
predicted = pipeline.predict(example)
print("예측된 사용장소:", predicted[0])

예측된 사용장소: 배수사


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 데이터 불러오기
df = pd.read_csv("프렌차이즈_구추출_결과.csv")

# 결측 처리
df['업종 중분류'] = df['업종 중분류'].fillna('기타')
df['구'] = df['구'].fillna('기타')

# 사용할 컬럼 정의
features = ['인원', '계절', '점저', '1인당비용', '업종 중분류', '구']
label = '사용장소'

# 결측값 제거
df_clean = df[features + [label]].dropna()

# 사용장소 기준 상위 10개만 필터링
top_places = df_clean[label].value_counts().nlargest(10).index
df_filtered = df_clean[df_clean[label].isin(top_places)]

# 특성과 타깃 분리
X = df_filtered[features]
y = df_filtered[label]

# 수치형/범주형 특성 정의
numeric_features = ['인원', '1인당비용']
categorical_features = ['계절', '점저', '업종 중분류', '구']

# 전처리 파이프라인
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# 전체 파이프라인 구성 (RandomForest 사용)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
pipeline.fit(X_train, y_train)

# 평가
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


In [18]:
# 예측 예시 입력
example = pd.DataFrame([{
    '인원': 7,
    '계절': '여름',
    '점저': '저녁',
    '1인당비용': 12000,
    '업종 중분류' : '일식',
    '구': '중구'
}])

# 예측 및 확률 계산
predicted_probs = pipeline.predict_proba(example)
predicted_class_index = predicted_probs.argmax()
predicted_place = pipeline.classes_[predicted_class_index]
confidence = predicted_probs[0][predicted_class_index]

# 예측 결과 출력
print("예측된 사용장소:", predicted_place)
print(f"신뢰도: {confidence:.2%}")

# 원본 데이터에서 해당 장소의 업종 중분류 조회
match = df[df['사용장소'] == predicted_place]
if not match.empty:
    category = match['업종 중분류'].iloc[0]
    print("업종 중분류:", category)

    # 동일 업종 중분류 내 비슷한 장소 3개 추천
    similar_places = df[(df['업종 중분류'] == category) & (df['사용장소'] != predicted_place)]
    top_similars = similar_places['사용장소'].value_counts().head(3).index.tolist()

    print("비슷한 장소 추천:")
    for place in top_similars:
        print("-", place)
else:
    print("해당 사용장소 정보가 원본 데이터에 없습니다.")


예측된 사용장소: 우도일식
신뢰도: 57.46%
업종 중분류: 일식
비슷한 장소 추천:
- 브이아이피참치
- 배수사
- 브이아이피참치 뉴서울호텔점


## 2. 음식점 데이터 필터링 및 전처리

In [3]:
df_food = df[df['업종 대분류'] == '음식점'].copy()
df_food = df_food.dropna(subset=['업종 중분류', '1인당비용', '계절', '점저', '인원'])
df_food


,집행일시,시간,집행목적,사용장소,인원,금액,계절,1인당비용,중복,주소지,업종 대분류,업종 중분류,업종 소분류,업종코드,점저
0,2023-10-31 0:00,오전 9:28:00,국가지정문화재 보수정비 현장검토 관련 간담회,연회다원,3,16500,가을,5500.0,2,강남구 봉은사로 531,음식점,카페,NaN,CE7,점심
1,2023-10-30 0:00,오후 12:14:00,부서 전입 직원 격려 간담회,한와담 광화문점,5,75000,가을,15000.0,130,중구 세종대로21길 40,음식점,한식,"육류,고기",FD6,점심
2,2023-10-27 0:00,오후 7:17:00,생생문화재 활용사업 추진 관련 간담회,무교동낙지,4,49000,가을,12250.0,653,중구 덕수궁길 7,음식점,한식,"해물,생선",FD6,저녁
3,2023-10-24 0:00,오후 12:33:00,문화재 지정조사 실시 관련 간담회,남산복집,4,60000,가을,15000.0,2,중구 무교동 1번지 신라 1호,음식점,한식,"해물,생선",FD6,점심
4,2023-10-23 0:00,오후 7:26:00,남산봉수의식 추진 관련 간담회,종로황소곱창,4,61000,가을,15250.0,2,종로구 우정국로 2길 31,음식점,한식,"육류,고기",FD6,저녁
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70680,2025-01-10 0:00,오전 11:52:00,가락시장 현대화사업 운영 관련 업무협의 간담회,전복집,8,153000,겨울,19125.0,31,중구 남대문로1길 30-5,음식점,한식,"해물,생선",FD6,점심
70681,2025-01-09 0:00,오후 12:11:00,서울먹거리창업센터 운영관련 간담회,오리마당,3,63000,겨울,21000.0,6,"중구 세종대로 92, 지하1층",음식점,한식,"육류,고기",FD6,점심
70682,2025-01-07 0:00,오후 8:20:00,부서 직원 격려 간담회 비용 지출,교동전선생,5,75000,겨울,15000.0,149,중구 서소문로 134-10,음식점,술집,"호프,요리주점",FD6,저녁
70683,2025-01-06 0:00,오후 12:41:00,농수산식품공사 주요 현안관련 간담회,어도횟집,5,54000,겨울,10800.0,4,서대문구 충정로6안길4,음식점,한식,"해물,생선",FD6,점심


## 3. 가격대 및 타겟 변수 생성

In [4]:
df_food['가격대'] = pd.qcut(df_food['1인당비용'], q=[0, 0.25, 0.75, 1], labels=['가성비', '중가', '고가'])
df_food['음식점_그룹'] = df_food['업종 중분류'] + '-' + df_food['가격대'].astype(str)
df_food


,집행일시,시간,집행목적,사용장소,인원,금액,계절,1인당비용,중복,주소지,업종 대분류,업종 중분류,업종 소분류,업종코드,점저,가격대,음식점_그룹
0,2023-10-31 0:00,오전 9:28:00,국가지정문화재 보수정비 현장검토 관련 간담회,연회다원,3,16500,가을,5500.0,2,강남구 봉은사로 531,음식점,카페,NaN,CE7,점심,가성비,카페-가성비
1,2023-10-30 0:00,오후 12:14:00,부서 전입 직원 격려 간담회,한와담 광화문점,5,75000,가을,15000.0,130,중구 세종대로21길 40,음식점,한식,"육류,고기",FD6,점심,중가,한식-중가
2,2023-10-27 0:00,오후 7:17:00,생생문화재 활용사업 추진 관련 간담회,무교동낙지,4,49000,가을,12250.0,653,중구 덕수궁길 7,음식점,한식,"해물,생선",FD6,저녁,가성비,한식-가성비
3,2023-10-24 0:00,오후 12:33:00,문화재 지정조사 실시 관련 간담회,남산복집,4,60000,가을,15000.0,2,중구 무교동 1번지 신라 1호,음식점,한식,"해물,생선",FD6,점심,중가,한식-중가
4,2023-10-23 0:00,오후 7:26:00,남산봉수의식 추진 관련 간담회,종로황소곱창,4,61000,가을,15250.0,2,종로구 우정국로 2길 31,음식점,한식,"육류,고기",FD6,저녁,중가,한식-중가
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70680,2025-01-10 0:00,오전 11:52:00,가락시장 현대화사업 운영 관련 업무협의 간담회,전복집,8,153000,겨울,19125.0,31,중구 남대문로1길 30-5,음식점,한식,"해물,생선",FD6,점심,중가,한식-중가
70681,2025-01-09 0:00,오후 12:11:00,서울먹거리창업센터 운영관련 간담회,오리마당,3,63000,겨울,21000.0,6,"중구 세종대로 92, 지하1층",음식점,한식,"육류,고기",FD6,점심,중가,한식-중가
70682,2025-01-07 0:00,오후 8:20:00,부서 직원 격려 간담회 비용 지출,교동전선생,5,75000,겨울,15000.0,149,중구 서소문로 134-10,음식점,술집,"호프,요리주점",FD6,저녁,중가,술집-중가
70683,2025-01-06 0:00,오후 12:41:00,농수산식품공사 주요 현안관련 간담회,어도횟집,5,54000,겨울,10800.0,4,서대문구 충정로6안길4,음식점,한식,"해물,생선",FD6,점심,가성비,한식-가성비


## 4. 모델 학습용 데이터 구성

In [5]:
features = ['인원', '계절', '점저', '1인당비용']
df_model = df_food.dropna(subset=['음식점_그룹'])
df_model = df_model[features + ['음식점_그룹', '사용장소']]

# 범주형 인코딩
le_season = LabelEncoder()
le_time = LabelEncoder()
df_model['계절'] = le_season.fit_transform(df_model['계절'])
df_model['점저'] = le_time.fit_transform(df_model['점저'])

# 최소 2개 이상 샘플 가진 클래스만 유지
value_counts = df_model['음식점_그룹'].value_counts()
valid_classes = value_counts[value_counts >= 2].index
df_model = df_model[df_model['음식점_그룹'].isin(valid_classes)]


## 5. 모델 학습 및 평가

In [6]:
X = df_model[features]
y = df_model['음식점_그룹']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

      간식-가성비       0.24      0.20      0.22       232
       간식-고가       0.00      0.00      0.00        10
       간식-중가       0.11      0.04      0.06        52
    구내식당-가성비       0.15      0.10      0.12        20
     구내식당-고가       0.22      0.21      0.22        19
     구내식당-중가       0.29      0.20      0.24        10
    기사식당-가성비       0.00      0.00      0.00         3
     도시락-가성비       0.18      0.16      0.17        19
      도시락-고가       0.00      0.00      0.00         1
      도시락-중가       0.00      0.00      0.00         6
      분식-가성비       0.12      0.05      0.07        96
       분식-고가       0.00      0.00      0.00         5
       분식-중가       0.12      0.03      0.05        30
      뷔페-가성비       0.00      0.00      0.00         8
       뷔페-고가       0.29      0.14      0.19        14
       뷔페-중가       0.44      0.25      0.32        48
     샐러드-가성비       0.00      0.00      0.00        35
      샐러드-고가       0.00    

c:\Users\User\anaconda3\envs\gongpick-env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\anaconda3\envs\gongpick-env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\anaconda3\envs\gongpick-env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()}

## 6. 예측 함수 정의 및 예시 입력

In [7]:
def predict_with_probability(model, input_data, df_source, topn=3):
    df_input = pd.DataFrame([input_data])
    pred_proba = model.predict_proba(df_input)[0]
    pred_index = pred_proba.argmax()
    pred_class = model.classes_[pred_index]
    confidence = pred_proba[pred_index]

    matched = df_source[df_source['음식점_그룹'] == pred_class]['사용장소'].value_counts().head(topn).index.tolist()
    return pred_class, confidence, matched
# 예시 입력
example_input = {
    '인원': 7,
    '계절': le_season.transform(['여름'])[0],
    '점저': le_time.transform(['점심'])[0],
    '1인당비용': 14000
}

pred_group, confidence, names = predict_with_probability(clf, example_input, df_model)
print("추천 카테고리:", pred_group)
print("추천 음식점:", names)
print("📊 예측 신뢰도:", round(confidence * 100, 2), "%")




추천 카테고리: 중식-중가
추천 음식점: ['배재반점', '복성각', '만복림']
📊 예측 신뢰도: 45.7 %
